In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
csv_file_path = 'fashion.csv'
df = pd.read_csv(csv_file_path)

train_df, val_df = train_test_split(df, test_size=0.25, random_state=42)

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

images_dir = 'images_with_product_ids'

In [6]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=images_dir,
    x_col='Image',
    y_col='SubCategory',
    target_size=(300, 300),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

Found 1744 validated image filenames belonging to 9 classes.


In [7]:
val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory=images_dir,
    x_col='Image',
    y_col='SubCategory',
    target_size=(300, 300),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 145 validated image filenames belonging to 9 classes.


In [9]:
def create_cnn_model(input_shape=(300, 300, 3), num_classes=9):
    base_model = VGG16(input_shape=input_shape, include_top=False, weights='imagenet')
    
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=x)
    return model

In [10]:
model = create_cnn_model()
model.compile(optimizer=Adam(learning_rate=0.001), loss=categorical_crossentropy, metrics=['accuracy'])

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 300, 300, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 300, 300, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 300, 300, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 150, 150, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 150, 150, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 150, 150, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 75, 75, 128)       0   

In [11]:
model.fit(train_generator, epochs=10, steps_per_epoch=len(train_generator), validation_data=val_generator, validation_steps=len(val_generator))


Epoch 1/10
55/55 [==============================] - 232s 4s/step - loss: 1.1978 - accuracy: 0.8394 - val_loss: 0.0991 - val_accuracy: 0.9655
Epoch 2/10
55/55 [==============================] - 234s 4s/step - loss: 0.1288 - accuracy: 0.9639 - val_loss: 0.0939 - val_accuracy: 0.9724
Epoch 3/10
55/55 [==============================] - 235s 4s/step - loss: 0.0452 - accuracy: 0.9862 - val_loss: 0.1614 - val_accuracy: 0.9517
Epoch 4/10
55/55 [==============================] - 236s 4s/step - loss: 0.0227 - accuracy: 0.9920 - val_loss: 0.1067 - val_accuracy: 0.9724
Epoch 5/10
55/55 [==============================] - 235s 4s/step - loss: 0.0076 - accuracy: 0.9989 - val_loss: 0.1094 - val_accuracy: 0.9724
Epoch 6/10
55/55 [==============================] - 236s 4s/step - loss: 0.0090 - accuracy: 0.9971 - val_loss: 0.0998 - val_accuracy: 0.9793
Epoch 7/10
55/55 [==============================] - 236s 4s/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.1259 - val_accuracy: 0.9793
Epoch 8/10
55

In [13]:
model.save('model.keras')